In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel
from pyspark.sql.types import *

In [2]:
df= spark.read.csv('/FileStore/tables/timetravel1.csv', inferSchema=True,header=True) #read sample data from location

In [3]:
df.printSchema() #verify schema

root
-- ID: integer (nullable = true)
-- Name: string (nullable = true)
-- Date: timestamp (nullable = true)
-- Sale: integer (nullable = true)
-- Type: string (nullable = true)
-- Coupon: string (nullable = true)

In [4]:
df= df.withColumn("Sale1", F.col("Sale").cast("float")).drop("Sale").persist(StorageLevel.MEMORY_AND_DISK) #changed Sale column from integer to float. New column: Sale1. AND persisted to memory for faster in-memory computations

# 6 new columns will be engineered to provide historical context and granularity to Sales. Sum over last4weeks, last13weeks, last52weeks. First three columns will be agg by type and coupon. Last three will agg by TOTAL SALES combined.

In [6]:
days = lambda i: i * 86400
w1 = (Window().partitionBy(F.col("Type"),F.col("Coupon")).orderBy(F.col("Date").cast('long')).rangeBetween(-days(27), 0))
w2 = (Window().partitionBy(F.col("Type"),F.col("Coupon")).orderBy(F.col("Date").cast('long')).rangeBetween(-days(90), 0))
w3 = (Window().partitionBy(F.col("Type"),F.col("Coupon")).orderBy(F.col("Date").cast('long')).rangeBetween(-days(364), 0))
w4 = (Window().orderBy(F.col("Date").cast('long')).rangeBetween(-days(27), 0))
w5 = (Window().orderBy(F.col("Date").cast('long')).rangeBetween(-days(90), 0))
w6 = (Window().orderBy(F.col("Date").cast('long')).rangeBetween(-days(364), 0))
df= df.withColumn("Sales-Last4weeks-by-C-S", F.sum("Sale1").over(w1))\
  .withColumn("Sales-Last13weeks-by-C-S", F.sum("Sale1").over(w2))\
  .withColumn("Sales-Last52weeks-by-C-S", F.sum("Sale1").over(w3))\
  .withColumn("Sales-Last4weeks-TOTAL", F.sum("Sale1").over(w4))\
  .withColumn("Sales-Last13weeks-TOTAL", F.sum("Sale1").over(w5))\
  .withColumn("Sales-Last52weeks-TOTAL", F.sum("Sale1").over(w6)).orderBy(df.Date.desc())
df.show(10)

+---+----+-------------------+-------+------+-----+-----------------------+------------------------+------------------------+----------------------+-----------------------+-----------------------+
 ID|Name| Date| Type|Coupon|Sale1|Sales-Last4weeks-by-C-S|Sales-Last13weeks-by-C-S|Sales-Last52weeks-by-C-S|Sales-Last4weeks-TOTAL|Sales-Last13weeks-TOTAL|Sales-Last52weeks-TOTAL|
+---+----+-------------------+-------+------+-----+-----------------------+------------------------+------------------------+----------------------+-----------------------+-----------------------+
 1| ETE|2019-08-29 00:00:00| Yearly| Yes| 95.0| 475.0| 570.0| 570.0| 475.0| 742.0| 1584.0|
 3| YH|2019-08-23 00:00:00| Yearly| Yes| 95.0| 380.0| 475.0| 475.0| 380.0| 647.0| 1489.0|
 2| TYT|2019-08-23 00:00:00| Yearly| Yes| 95.0| 380.0| 475.0| 475.0| 380.0| 647.0| 1489.0|
 4| FG|2019-08-12 00:00:00| Yearly| Yes| 95.0| 285.0| 285.0| 285.0| 285.0| 472.0| 1399.0|
 5| FFF|2019-08-12 00:00:00| Yearly| Yes| 95.0| 285.0| 285.0| 285.0| 285.0| 472.0| 1399.0|
 6| GGG|2019-07-24 00:00:00| Yearly| Yes| 95.0| 95.0| 95.0| 95.0| 119.0| 297.0| 1209.0|
 7|GUMU|2019-07-12 00:00:00|Monthly| Yes| 12.0| 48.0| 84.0| 84.0| 148.0| 214.0| 1114.0|
 8| GHU|2019-07-11 00:00:00|Monthly| Yes| 12.0| 36.0| 72.0| 72.0| 136.0| 202.0| 1102.0|
 9| GHG|2019-06-21 00:00:00|Monthly| Yes| 12.0| 48.0| 60.0| 60.0| 148.0| 190.0| 1090.0|
 16| RTG|2019-06-21 00:00:00| Yearly| No|100.0| 100.0| 100.0| 1000.0| 148.0| 190.0| 1090.0|
+---+----+-------------------+-------+------+-----+-----------------------+------------------------+------------------------+----------------------+-----------------------+-----------------------+
only showing top 10 rows

In [7]:
df.createOrReplaceTempView("timetravel")

In [8]:
spark.sql("Select * from timetravel where Date Between '2019-06-01' AND '2019-09-01'").show()

+---+----+-------------------+-------+------+-----+-----------------------+------------------------+------------------------+----------------------+-----------------------+-----------------------+
 ID|Name| Date| Type|Coupon|Sale1|Sales-Last4weeks-by-C-S|Sales-Last13weeks-by-C-S|Sales-Last52weeks-by-C-S|Sales-Last4weeks-TOTAL|Sales-Last13weeks-TOTAL|Sales-Last52weeks-TOTAL|
+---+----+-------------------+-------+------+-----+-----------------------+------------------------+------------------------+----------------------+-----------------------+-----------------------+
 1| ETE|2019-08-29 00:00:00| Yearly| Yes| 95.0| 475.0| 570.0| 570.0| 475.0| 742.0| 1584.0|
 2| TYT|2019-08-23 00:00:00| Yearly| Yes| 95.0| 380.0| 475.0| 475.0| 380.0| 647.0| 1489.0|
 3| YH|2019-08-23 00:00:00| Yearly| Yes| 95.0| 380.0| 475.0| 475.0| 380.0| 647.0| 1489.0|
 4| FG|2019-08-12 00:00:00| Yearly| Yes| 95.0| 285.0| 285.0| 285.0| 285.0| 472.0| 1399.0|
 5| FFF|2019-08-12 00:00:00| Yearly| Yes| 95.0| 285.0| 285.0| 285.0| 285.0| 472.0| 1399.0|
 6| GGG|2019-07-24 00:00:00| Yearly| Yes| 95.0| 95.0| 95.0| 95.0| 119.0| 297.0| 1209.0|
 7|GUMU|2019-07-12 00:00:00|Monthly| Yes| 12.0| 48.0| 84.0| 84.0| 148.0| 214.0| 1114.0|
 8| GHU|2019-07-11 00:00:00|Monthly| Yes| 12.0| 36.0| 72.0| 72.0| 136.0| 202.0| 1102.0|
 9| GHG|2019-06-21 00:00:00|Monthly| Yes| 12.0| 48.0| 60.0| 60.0| 148.0| 190.0| 1090.0|
 16| RTG|2019-06-21 00:00:00| Yearly| No|100.0| 100.0| 100.0| 1000.0| 148.0| 190.0| 1090.0|
 10|HOOH|2019-06-20 00:00:00|Monthly| Yes| 12.0| 36.0| 48.0| 48.0| 36.0| 78.0| 978.0|
 11| PQ|2019-06-05 00:00:00|Monthly| Yes| 12.0| 24.0| 36.0| 36.0| 39.0| 166.0| 966.0|
 12| WE|2019-06-01 00:00:00|Monthly| Yes| 12.0| 12.0| 24.0| 24.0| 27.0| 154.0| 954.0|
+---+----+-------------------+-------+------+-----+-----------------------+------------------------+------------------------+----------------------+-----------------------+-----------------------+